#### Este baseline se basara en armar un modelo LSTM por cada producto, con una optimizacion de hiper parametros escueta, para poder comparar con futuros experimientos. En caso de que esta alternativa funcione bien, seria recomendable incorporar parametros de optimizacion extra.

#### Imports

In [13]:
import warnings
warnings.filterwarnings('ignore')
from fbprophet import Prophet

import pandas as pd
import numpy as np

In [14]:
final_dataset = pd.read_csv('../../Datasets/final_dataset.csv', sep='\t')

In [15]:
final_dataset.head()

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age
0,201701,20001,0,479,937.72717,934.77222,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0
1,201702,20001,0,432,833.72187,798.01620,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,1
2,201703,20001,0,509,1330.74697,1303.35771,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,1,2
3,201704,20001,0,279,1132.94430,1069.96130,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,3
4,201705,20001,0,701,1550.68936,1502.20132,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,4


In [16]:
columns = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'close_quarter','y']

#### Funcion para crear los modelos

#### Armado de los modelos

In [17]:
import os
from pmdarima import auto_arima
import joblib

product_ids = final_dataset['product_id'].unique()
predictions = []
    
for product_id in product_ids:
    product_data = final_dataset[final_dataset['product_id'] == product_id].sort_values(by='periodo')[columns]
    
    product_data_array = product_data['y'].values
    
    model = auto_arima(product_data_array, n_fits=50, trace=True, error_action='ignore', suppress_warnings=True)

    os.makedirs('Arima_results', exist_ok=True)
    joblib.dump(model, f'Models_params/model_product_{product_id}.pkl')
    
    forecast_steps = 2
    forecast = model.predict(n_periods=forecast_steps)
    
    # Agregar predicción al resultado
    predicted_y = forecast[-1]
    predictions.append({'product_id': product_id, 'predicted_y': predicted_y})

    print(f'Modelo para el producto {product_id} entrenado y guardado. Predicción a 2 meses: {predicted_y}')

 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=105.414, Time=1.76 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=101.783, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=103.679, Time=0.05 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=103.287, Time=0.50 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=135.005, Time=0.01 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.80 sec

Best model:  ARIMA(0,0,0)(0,0,0)[0] intercept
Total fit time: 3.138 seconds
Modelo para el producto 20527 entrenado y guardado. Predicción a 2 meses: 14.568020625000006
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=inf, Time=2.74 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=95.582, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=96.002, Time=0.06 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=inf, Time=1.18 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=134.392, Time=0.01 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=inf, Time=2.57 sec

Best model:  ARIMA(0,0,0

In [19]:
predictions_df = pd.DataFrame(predictions)
predictions_df.to_csv('../../Datasets/predictions.csv', index=False)

print('Todas las predicciones han sido generadas y guardadas en predictions.csv.')

Todas las predicciones han sido generadas y guardadas en predictions.csv.
